In [3]:
import pandas as pd
import boto3
import json

In [4]:
import configparser
config = configparser.ConfigParser()
config.read_file(open('dwh.cfg'))

KEY                    = config.get('AWS','KEY')
SECRET                 = config.get('AWS','SECRET')

DWH_CLUSTER_TYPE       = config.get("DWH","DWH_CLUSTER_TYPE")
DWH_NUM_NODES          = config.get("DWH","DWH_NUM_NODES")
DWH_NODE_TYPE          = config.get("DWH","DWH_NODE_TYPE")

DWH_CLUSTER_IDENTIFIER = config.get("DWH","DWH_CLUSTER_IDENTIFIER")
DWH_DB                 = config.get("DWH","DWH_DB")
DWH_DB_USER            = config.get("DWH","DWH_DB_USER")
DWH_DB_PASSWORD        = config.get("DWH","DWH_DB_PASSWORD")
DWH_PORT               = config.get("DWH","DWH_PORT")

DWH_IAM_ROLE_NAME      = config.get("DWH", "DWH_IAM_ROLE_NAME")

(DWH_DB_USER, DWH_DB_PASSWORD, DWH_DB)

pd.DataFrame({"Param":
                  ["DWH_CLUSTER_TYPE", "DWH_NUM_NODES", "DWH_NODE_TYPE", "DWH_CLUSTER_IDENTIFIER",
                   "DWH_DB", "DWH_DB_USER", "DWH_DB_PASSWORD", "DWH_PORT", "DWH_IAM_ROLE_NAME"],
              
              "Value":
                  [DWH_CLUSTER_TYPE, DWH_NUM_NODES, DWH_NODE_TYPE, DWH_CLUSTER_IDENTIFIER,
                   DWH_DB, DWH_DB_USER, DWH_DB_PASSWORD, DWH_PORT, DWH_IAM_ROLE_NAME]
             })

,Param,Value
0,DWH_CLUSTER_TYPE,multi-node
1,DWH_NUM_NODES,4
2,DWH_NODE_TYPE,dc2.large
3,DWH_CLUSTER_IDENTIFIER,dwhCluster-sparkify
4,DWH_DB,dwh_sparkify
5,DWH_DB_USER,dwhuser_sparkify
6,DWH_DB_PASSWORD,Passw0rd
7,DWH_PORT,5439
8,DWH_IAM_ROLE_NAME,dwhRole_sparkify


In [5]:
s3 = boto3.resource('s3',
                  region_name="us-west-2",
                  aws_access_key_id=KEY,
                  aws_secret_access_key=SECRET
                  )

In [6]:
bucket =  s3.Bucket("udacity-dend")

# Iterating over bucket objects, printing them if they start with "ssbgz"
# for page in sampleDbBucket.objects.pages():
#      for obj in page:
#             if obj.key[:5] == 'song':
#                 print(obj.key)

In [7]:
f = [filename for filename in bucket.objects.filter(Prefix='song-data/A/A').limit(10)]
f

[s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAK128F9318786.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAAV128F421A322.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAABD128F429CF47.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAACN128F9355673.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEA128F935A30D.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAED128E0783FAB.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEM128F93347B9.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAEW128F42930C0.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAFD128F92F423A.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='song-data/A/A/A/TRAAAGR128F425B14B.json')]

In [28]:
obj = s3.Object("udacity-dend", f[0].key)
body = obj.get()['Body'].read()
dict_str = body.decode("UTF-8")
dict_str

'{"song_id": "SOBLFFE12AF72AA5BA", "num_songs": 1, "title": "Scream", "artist_name": "Adelitas Way", "artist_latitude": null, "year": 2009, "duration": 213.9424, "artist_id": "ARJNIUY12298900C91", "artist_longitude": null, "artist_location": ""}'

In [35]:
d = {"song_id": "SOBLFFE12AF72AA5BA", "num_songs": 1, "title": "Scream", "artist_name": "Adelitas Way", 
      "artist_latitude": None, "year": 2009, "duration": 213.9424, "artist_id": "ARJNIUY12298900C91",
      "artist_longitude": None, "artist_location": ""}

In [26]:
pd.DataFrame(data=d, index=[0])

,song_id,num_songs,title,artist_name,artist_latitude,year,duration,artist_id,artist_longitude,artist_location
0,SOBLFFE12AF72AA5BA,1,Scream,Adelitas Way,None,2009,213.9424,ARJNIUY12298900C91,None,


In [31]:
g = [filename for filename in bucket.objects.filter(Prefix='log').limit(10)]
g

[s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-01-events.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-02-events.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-03-events.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-04-events.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-05-events.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-06-events.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-07-events.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-08-events.json'),
 s3.ObjectSummary(bucket_name='udacity-dend', key='log-data/2018/11/2018-11-09-events.json')]

In [45]:
obj = s3.Object("udacity-dend", g[1].key)
body = obj.get()['Body'].read()
dict_str = body.decode("UTF-8")
dict_str

'{"artist":null,"auth":"Logged In","firstName":"Walter","gender":"M","itemInSession":0,"lastName":"Frye","length":null,"level":"free","location":"San Francisco-Oakland-Hayward, CA","method":"GET","page":"Home","registration":1540919166796.0,"sessionId":38,"song":null,"status":200,"ts":1541105830796,"userAgent":"\\"Mozilla\\/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit\\/537.36 (KHTML, like Gecko) Chrome\\/36.0.1985.143 Safari\\/537.36\\"","userId":"39"}\n{"artist":null,"auth":"Logged In","firstName":"Kaylee","gender":"F","itemInSession":0,"lastName":"Summers","length":null,"level":"free","location":"Phoenix-Mesa-Scottsdale, AZ","method":"GET","page":"Home","registration":1540344794796.0,"sessionId":139,"song":null,"status":200,"ts":1541106106796,"userAgent":"\\"Mozilla\\/5.0 (Windows NT 6.1; WOW64) AppleWebKit\\/537.36 (KHTML, like Gecko) Chrome\\/35.0.1916.153 Safari\\/537.36\\"","userId":"8"}\n{"artist":"Des\'ree","auth":"Logged In","firstName":"Kaylee","gender":"F","itemInSess